# Linkedin Cookies Updater

# Input

## Import Libraries

In [1]:
import boto3
import naas
import pydash

import escapism
import string

import json
import base64
import uuid
import datetime

from dataclasses import dataclass, field
try:
    from dataclasses_json import dataclass_json, config
except:
    !pip install --user dataclasses-json
    from dataclasses_json import dataclass_json, config


## Define globals

In [2]:
SQS_QUEUE_URL = 'https://sqs.eu-west-3.amazonaws.com/903885477968/linkedin-cookies-production.fifo'

## Configure SQS Client

In [3]:
sqs = boto3.client(
    'sqs',
                   region_name="eu-west-3",
                   aws_access_key_id=naas.secret.get('LINKEDIN_COOKIES_UPDATER_AWS_ACCESS_KEY_ID'),
                aws_secret_access_key=naas.secret.get('LINKEDIN_COOKIES_UPDATER_AWS_SECRET_ACCESS_KEY'))

# Model

## Create helper functions

In [4]:
def escape(s):
    return escapism.escape(
    s, 
    safe=set(string.ascii_letters + string.digits), 
    escape_char='-').lower()

## Create Helper class

In [5]:
@dataclass_json
@dataclass
class EventContent:
    li_at: str
    jsessionid: str

@dataclass_json
@dataclass
class Event:
    event_type: str
    event_content: EventContent

@dataclass_json
@dataclass
class SQSMessage:
    version: int
    id: str
    detail_type: str = field(metadata=config(field_name="detail-type"))
    source: str
    account: str
    time: str
    region: str
    detail: Event

## Create SQS functions

In [6]:
def get_sqs_messages(max_number_of_messages:int=1):
    response = sqs.receive_message(
        AttributeNames=['All'],
        QueueUrl=SQS_QUEUE_URL,
        WaitTimeSeconds=3,
        MaxNumberOfMessages=max_number_of_messages
    )
    return response['Messages'] if 'Messages' in response else []

def delete_sqs_message(receipt_handle: str):
    return sqs.delete_message(
        QueueUrl=SQS_QUEUE_URL,
        ReceiptHandle=receipt_handle
    )

## Read, Encode and Decode secrets

In [7]:
def decode(secret_base64):
    secret = base64.b64decode(secret_base64)
    secret_decoded = secret.decode("ascii")
    return secret_decoded

def encode(text):
    message_bytes = text.encode("ascii")
    base64_bytes = base64.b64encode(message_bytes)
    secret_base64 = base64_bytes.decode("ascii")
    return secret_base64

def update_secret(target_file, key, value):
    content = None
    try:
        with open(target_file, 'r') as f:
            content = json.load(f)
            f.close()
    except:
        content = []
    
    
    r = pydash.find(content, lambda x: x['name'] == key)
    now = datetime.datetime.now()
    dt_string = now.strftime("%Y-%m-%d %H:%M:%S")
    if r != None:
        r['lastUpdate'] = dt_string
        r['secret'] = encode(value)
    else:
        content.append({
            "id": str(uuid.uuid4()),
            "name": key,
            "secret": encode(value),
            "lastUpdate": dt_string,
        })

    with open(target_file, 'w+') as wfile:
        wfile.write(
            json.dumps(content, sort_keys=True, indent=4).replace("NaN", "null")
        )
        wfile.close()
        

## Handle an SQS message

In [8]:
def handle_sqs_message(message):
    # Get message  content
    message_content = SQSMessage.from_json(message['Body'])
    
    # Get message type
    message_type = message_content.detail_type
    
    # Make sure we have the right type
    if message_type != 'linkedin.cookies':
        print(message_content)
        raise Exception('This message should not be in this SQS queue.')
        
    # Get the user to which this message belongs to.
    user = message_content.source
    
    print(f'💡 Handling cookies for user {user}')
    
    # Get linkedin cookies.
    li_at = message_content.detail.event_content.li_at
    jsessionid = message_content.detail.event_content.jsessionid.replace('"', '')
    
    print(f'LINKEDIN_LI_AT = {li_at}')
    print(f'LINKEDIN_JSESSIONID = {jsessionid}')
    
    # Compute path to user's secrets file
    user_secrets_path = f'/home/ftp/efs/dev/ftpusers/{escape(user)}/.naas/secrets.json'
    
    # Update secrets
    update_secret(user_secrets_path, 'LINKEDIN_LI_AT', li_at)
    update_secret(user_secrets_path, 'LINKEDIN_JSESSIONID', jsessionid)
    
    # Delete SQS message
    delete_sqs_message(message['ReceiptHandle'])
    
    return user
    

## Handle all messages

In [9]:
def handle_all_messages():
    while True:
        messages = get_sqs_messages()
        if len(messages) == 0:
            break
        for message in messages:
            handle_sqs_message(message)

# Output

In [10]:
handle_all_messages()

In [11]:
naas.scheduler.add(cron="* * * * *")

<IPython.core.display.Javascript object>

👌 Well done! Your Notebook has been sent to production.

⏰ It will be scheduled "Every minute of every day" (more on the syntax on https://crontab.guru/).

Ps: to remove the "Scheduler", just replace .add by .delete
